In [ ]:
import pandas as pd

citibike2016_df = pd.DataFrame()

#combine all trip data CSVs of 2016
for year in [2016]:
    for month in range(1,13):
        df = pd.read_csv('https://s3.amazonaws.com/tripdata/'+str(year)+str(month).zfill(2)+'-citibike-tripdata.zip', compression='zip', header=0, sep=',', quotechar='"')
        citibike2016_df = citibike2016_df.append(df)
        

In [ ]:
citibike2016_df.head()